#### Just for experimentation, use train_lstm.py!!!

In [1]:
# This Python file uses the following encoding: utf-8
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

from lib.pretty_testing import predict_test

import numpy as np
import pickle

Using TensorFlow backend.


In [2]:
dataset_file = 'embedded_docs_with_verb.p'
model_file = 'models/keras_model.json'
model_weights_file = 'models/keras_new_weights_with_verb_es.h5'

In [3]:
# fix random seed for reproducibility
np.random.seed(666)


# load prepared data
with open(dataset_file) as f:
    data, labels = pickle.load(f)

In [4]:
# padding for the rnn
padded_data = sequence.pad_sequences(data, maxlen=200,padding="pre", truncating="post", value=0.0, dtype='float32')

In [5]:
# free ram from the original dataset
del data

In [7]:
# split dataset
X_train, X_test, y_train, y_test = train_test_split(padded_data, labels, train_size=0.9, stratify=labels)
del padded_data

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape = (200, 100), return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

'''
model = Sequential()
model.add(LSTM(100, input_shape = (200, 100)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=lr), metrics=['accuracy'])
print(model.summary())
'''


In [ ]:
with open(model_file,'w') as f:
    f.write(model.to_json())

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=64, )
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# serialize weights to HDF5
model.save_weights(model_weights_file)

### Test loading

In [8]:
with open('models/keras_model.json') as f:
    model = model_from_json(f.read())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 80,501
Trainable params: 80,501
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.load_weights("models/keras_new_weights_with_verb_es.h5")

In [43]:
predict_test(model, X_test, y_test, ["non_cost", "cost"])

()
Test classification report
Accuracy: 0.871552
             precision    recall  f1-score   support

          0       0.93      0.80      0.86      1903
          1       0.82      0.94      0.88      1904

avg / total       0.88      0.87      0.87      3807

Test confusion Matrix
             non_cost     cost
    non_cost   1520.0    383.0
        cost    106.0   1798.0


array([[0],
       [1],
       [0],
       ..., 
       [1],
       [0],
       [0]], dtype=int32)

In [55]:
v = model.layers[1]
v.get_weights()

[array([[ 0.02028171],
        [ 0.18235052],
        [ 0.23812914],
        [-0.36414918],
        [ 0.21093391],
        [ 0.08250543],
        [-0.20228331],
        [-0.30423039],
        [ 0.43490878],
        [-0.13503398],
        [-0.17565024],
        [-0.13670991],
        [ 0.31127021],
        [ 0.33547062],
        [-0.31177467],
        [ 0.35819039],
        [ 0.41784394],
        [-0.40707791],
        [ 0.25223097],
        [ 0.01939964],
        [ 0.34188497],
        [-0.16156748],
        [-0.36896807],
        [ 0.21450199],
        [ 0.34476218],
        [-0.19144231],
        [ 0.08315672],
        [ 0.25255978],
        [-0.18793416],
        [ 0.25187981],
        [ 0.23727795],
        [ 0.18443318],
        [-0.0417008 ],
        [-0.04900378],
        [ 0.0998246 ],
        [ 0.2002323 ],
        [ 0.35828298],
        [-0.17253509],
        [ 0.28414303],
        [-0.16518   ],
        [ 0.53428423],
        [-0.25924474],
        [-0.12657176],
        [-0

# Test

In [22]:
# This Python file uses the following encoding: utf-8
from lib.words import tokenize_doc
from lib.embedding import reduce_dictionary, sentence_vector
from gensim.models import Doc2Vec
import lib.text_extraction as te
from functools import partial

import codecs
import json
import os
import glob
import sys

In [23]:
labels_map = ['NON COSTITUTIVO', 'COSTITUTIVO']

pdf_folder = '../files_to_predict'
pdf_names = glob.glob('../files_to_predict/*')

# For OCR...
png_dir = '../tmp'
min_words = 150
pages = 5

In [19]:
# Accetto solo file con dimensione < 5MB
def file_filter(f):
    return os.path.getsize(str(f.absolute()))<(1024**2*5)
    #return f.name.endswith(u'001.pdf') and os.path.getsize(str(f.absolute()))<(1024**2*5)

def embed_document(model, doc, permitted_words):
    return [sentence_vector(model, sentence, permitted_words) for sentence in doc]

def predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words):
    filtered_filenames = [f for f,t in zip(filenames, txts) if (t != None and len(t)>0)]
    not_empty_txts =  [t for t in txts if (t != None and len(t)>0)]
    
    splitted_txts = [tokenize_doc(txt) for txt in not_empty_txts] 
    filtered_txts = [list(reduce_dictionary(document, permitted_words)) for document in splitted_txts]
    embedded_txts = [embed_document(gensim_model, doc, permitted_words) for doc in filtered_txts]
    padded_data = sequence.pad_sequences(embedded_txts, maxlen=200, padding="pre", truncating="post", value=0.0, dtype='float32')
    probs = keras_model.predict_proba(padded_data, verbose=0)
    return [prob[0] for prob in probs], filtered_filenames

def predict_documents_pdf(filenames, gensim_model, keras_model, permitted_words, do_ocr=False):
    txts = [te.extract_text(filename, do_ocr, png_dir, min_words, pages) for filename in filenames]
    return predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words)

def predict_documents_txt(filenames, gensim_model, keras_model, permitted_words):
    txts = [open(filename).read() for filename in filenames]
    return predict_documents_str(filenames, txts, gensim_model, keras_model, permitted_words)

In [39]:
models = {}
models['gensim_model'] = Doc2Vec.load('models/gensim_5000_model_with_verb.d2v')
models['keras_model'] = model

with open('first_5000_words_with_verb_cost.json') as f:
    models['permitted_words'] = set(json.load(f))
    
#predict_pdfs = partial(predict_documents_pdf, gensim_model=gensim_model, keras_model=keras_model, permitted_words=permitted_words)
predict_pdfs = partial(predict_documents_pdf, **models)

In [40]:
predictions, filtered_filenames = predict_pdfs(pdf_names)

In [41]:
for name, pred in zip(filtered_filenames, predictions):
    print os.path.basename(name)[:50].ljust(50), '\t-\t', labels_map[int(round(pred))], ' (%f)' % pred

3297438520001.pdf                                  	-	NON COSTITUTIVO  (0.166989)
4908019380001.pdf                                  	-	COSTITUTIVO  (0.956963)
3297448050002.pdf                                  	-	NON COSTITUTIVO  (0.003005)
4907876150001.pdf                                  	-	COSTITUTIVO  (0.970436)
4907903520001.pdf                                  	-	COSTITUTIVO  (0.995283)
4907874650001.pdf                                  	-	COSTITUTIVO  (0.986870)
3296082450001.pdf                                  	-	NON COSTITUTIVO  (0.020570)
4908013560001.pdf                                  	-	COSTITUTIVO  (0.937010)
4907882890001.pdf                                  	-	COSTITUTIVO  (0.982441)
4907924550001.pdf                                  	-	COSTITUTIVO  (0.596382)
4910203190001.pdf                                  	-	COSTITUTIVO  (0.973300)
4907941200001.pdf                                  	-	COSTITUTIVO  (0.959041)
4907932630001.pdf                                  	